In [ ]:
source_files = [
    'LLAMA-2-13b_P1R.json'
]

In [ ]:
import json

def convert_response(response):
    # для P1o
    mapping = {
        '(A)': '1',
        '(B)': '2',
        '(C)': '3',
        '(D)': '4',
        '(E)': '5'
    }
    # для P1R
    # mapping = {
    #     '(A)': '5',
    #     '(B)': '4',
    #     '(C)': '3',
    #     '(D)': '2',
    #     '(E)': '1'
    # }
    # для P2o, P3o
    # mapping = {
    #     '5': '5',
    #     '4': '4',
    #     '3': '3',
    #     '2': '2',
    #     '1': '1'
    # }
    # для P2R, P3R
    # mapping = {
    #     '1': '5',
    #     '2': '4',
    #     '3': '3',
    #     '4': '2',
    #     '5': '1'
    # }

    # Ищем буквенный или числовой код ответа
    for letter_code, number in mapping.items():
        if response:
            if response.startswith(letter_code):
                return number

    return response

# Загрузка данных
for x in source_files:
    with open(f"cleaned_data/{x}", 'r', encoding='utf-8') as f:
        data = json.load(f)

# Обработка каждого ответа
    for item in data:
        if 'processed_response' in item:
            item['processed_response'] = convert_response(item['processed_response'])

# Сохранение результата
    with open(f'cleaned_and_conv_data/{x}', 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=2)

In [ ]:
import json
import csv
import os
from collections import defaultdict

json_files = [
    '/content/cleaned_and_conv_data/LLAMA-2-13b_P1O.json',
    '/content/cleaned_and_conv_data/LLAMA-2-13b_P1R.json',
    '/content/cleaned_and_conv_data/LLAMA-2-13b_P2O.json',
    '/content/cleaned_and_conv_data/LLAMA-2-13b_P2R.json',
    '/content/cleaned_and_conv_data/LLAMA-2-13b_P3O.json',
    '/content/cleaned_and_conv_data/LLAMA-2-13b_P3R.json'
]

for file in json_files:
    if not os.path.exists(file):
        raise FileNotFoundError(f"Файл {file} не найден")

# Порядок сортировки меток
label_order = ['O', 'C', 'E', 'A', 'N']

# Словарь для хранения данных
position_data = defaultdict(lambda: defaultdict(dict))

# Читаем данные из всех файлов
for file in json_files:
    file_key = os.path.splitext(file)[0][-3:]

    with open(file, 'r', encoding='utf-8') as f:
        data = json.load(f)

        for position, item in enumerate(data):
            label = item['label']
            response = item['processed_response']
            position_data[position][label][file_key] = response

# Собираем все метки для сортировки
all_labels = []
for pos_data in position_data.values():
    all_labels.extend(pos_data.keys())
all_labels = sorted(set(all_labels), key=lambda x: label_order.index(x))

# Создаем CSV файл
output_file = 'combined_results_all.csv'
with open(output_file, 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile)
    headers = ['Label'] + [os.path.splitext(f)[0][-3:] for f in json_files]
    writer.writerow(headers)
    # Записываем данные, сортируя по меткам
    for label in sorted(all_labels, key=lambda x: label_order.index(x)):
        for position, pos_data in position_data.items():
            if label in pos_data:
                row = [label]
                for file in json_files:
                    file_key = os.path.splitext(file)[0][-3:]
                    row.append(pos_data[label].get(file_key, ''))
                writer.writerow(row)

print(f"Результат сохранен в {output_file}, всего строк: {sum(len(v) for v in position_data.values())}")